# Retraining Model for Classification Model


#### This notebook collects the validated data from cosmosdb of the last month if any starting date and ending date is not provided,and then based on the corrected annotation crops the images into two folder cfu and mold which is getting saved inside data_folder/training_images and from their it is splitted into train and test with five randomly suffeled set.
#### Then it takes a list of models after that it will train all the model with data for each iteration and will store the metrices in detailed _acuracy and model_score dictionary and after that it will also monitor  the existing model .Then a word document will be saved in the current directory with all the detailed accuracy figure .

In [ ]:
import sys

In [ ]:
sys.path.append("../src/")

In [ ]:
%load_ext dotenv
%dotenv ../services/environments/.env.secrets

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
%load_ext snakeviz

In [ ]:
#importing the neccessary packages 
import glob
from docx import Document
import cv2
import shutil
import json
import PIL
import tensorflow as tf
import pathlib
import math
from shutil import copyfile
from os import path
import numpy as np
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50,ResNet101
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import DenseNet121
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from PIL import Image, ImageOps
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import random
from skimage.util import random_noise
import splitfolders
import os
import sklearn
from sklearn import metrics
from sklearn.metrics import classification_report,f1_score, accuracy_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from statistics import mean
import sys
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.database import Database
from urllib.parse import quote_plus
from datetime import datetime,timedelta
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt 

In [ ]:
%matplotlib inline

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from vxgioq_cfu_counting_ml.api.data_models import Predictions, Region, ShapeAttributes
from vxgioq_cfu_counting_ml.training.dataset_mapper import CFUDatasetMapper
from vxgioq_cfu_counting_ml.training.data_analysis import get_dataset_stats, check_retest_models, compute_kpis, get_data_compute_predictions_save_zip_upload_images_names_predictions_count, construct_model_info, predict_images
from vxgioq_cfu_counting_ml.training.matched_data import MATCHES
from vxgioq_cfu_counting_ml.training.organize_data import get_paths_test_images, get_paths_train_images, count_files, get_paths_positive_unlabelled_images, via_to_detectron2, custom_train_test_split, generate_train_test, gsk_original_to_detectron2, negative_to_detectron2
from vxgioq_cfu_counting_ml.training.trainer import CFUTrainer

from vxgioq_cfu_counting_ml.utils.azure import (
    download_image,
    delete_object_from_blob,
    download_load_models_kpis,
    download_unzip_clean_zip,
    get_cosmos_mongo_collection,
    get_cosmos_mongo_collection_phase2,
    get_file_from_blob,
    list_data_on_blob,
    list_models_on_blob,
    upload_file_to_blob,
    download_unzip_clean_data,
    download_unzip_clean_model,
    write_zip_upload_models_kpis,
    zip_upload_data_to_blob,
    zip_upload_file_folder_to_blob,
    zip_upload_model_to_blob,
    zip_upload_predictions_to_blob,
    ACCOUNT_URL,
    CONTAINER,
    DATASET_DETECTRON2_NAME,
    DATASETS_QUANTILES_NAME,
    MODEL_CONFIG_NAME,
    MODELS_KPIS_NAME,
    MODELS_QUANTILES_NAME,
    MODEL_WEIGHTS_NAME,
    PATH_DATA_ON_DISK,
    PATH_MODELS_ON_DISK,
    PATH_PREDICTIONS_ON_DISK
)
from vxgioq_cfu_counting_ml.utils.conf import Backbone, Device, clean_output_dir_add_cfg, delete_files, delete_folders, prepare_cfu_detectron2_config_from_detectron2_model_zoo, prepeare_cfu_detectron_config_from_existing_model
from vxgioq_cfu_counting_ml.utils.prediction_intervals import pad_arrays, scores_to_distributions, generate_pi_dataset, DataType, train_quantile_regressor, HPSpace, verify_quantiles_predictions, BinningOption, DistributionFunctionType, scores_to_compiled_cdfs
from vxgioq_cfu_counting_ml.utils.convert import enhance_detectron2_dataset, extract_challenging_images, load_scan4000_excel, extract_png_from_scan4000_report, extract_metadata_from_scan4000_report, extract_all_from_scan4000_report
from vxgioq_cfu_counting_ml.utils.custom_datasets import func_dataset, mask_2_polygon
from vxgioq_cfu_counting_ml.utils.display import compare_scan4000_and_dl_images, display_batch, display_pi, display_dataset, display_enhanced_dataset, display_features_importance, display_violin, NormalizeOption, save_examples
from vxgioq_cfu_counting_ml.utils.evaluator import CFUEvaluator
from vxgioq_cfu_counting_ml.utils.loader import load_model, get_best_model_name, get_best_model, load_images
from vxgioq_cfu_counting_ml.utils.metrics import mse, mae, mape, calibration
from vxgioq_cfu_counting_ml.utils.predictor import CFUPredictor
from vxgioq_cfu_counting_ml.utils.types import *


In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.models import Model
from os import path

## Deleting existing data folder and new_data folder created during previous run

In [ ]:
#removing processed data
dir_del=PATH_DATA_ON_DISK+'/data_folder'
try:
    shutil.rmtree(dir_del)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))


In [ ]:
#removing real data
dir_del=PATH_DATA_ON_DISK+'/new_data'
try:
    shutil.rmtree(dir_del)
except OSError as e:
    print("Error: %s - %s." % (e.filename, e.strerror))

## Collecting Only the validated data.

#### We are  collecting data from cosmosdb which are validated by customer,user also can select a particular time period  for which last_month variable has to be set as False and give the date range otherwise it will collect last month data by default.Also We  can  limit the no of images we want to collect for which we have to give desired no in no_of_images

In [ ]:
dishes_collection2 = get_cosmos_mongo_collection_phase2()

In [ ]:
last_month=True #if we need images from the past 30 days
no_of_images =1000 #the number of images selected
days = 90 #number of days
no_of_days = timedelta(days)

In [ ]:
if(last_month):
    #selects the start date 30 days before the current date
    ending_date = datetime.now()
    starting_date = ending_date - no_of_days

else:
    #for a particular timespan
    starting_date = datetime(2021,10,16) # year,month,day format
    ending_date = starting_date + no_of_days
start_date = starting_date.strftime('%Y-%m-%dT%H:%M:%S.%sZ')
end_date = ending_date.strftime('%Y-%m-%dT%H:%M:%S.%sZ')

In [ ]:
total_jsons = list(dishes_collection2.find({'predictions.image.predictions_datetime':{"$gte":starting_date, "$lte":ending_date}}))

print(len(total_jsons))

processed_images=len(total_jsons)

In [ ]:
dishes = list(dishes_collection2.find({ "$and": [ { "is_validated":1 }, { 'corrections.image.validated_datetime': {"$gte":start_date, "$lte":end_date} } ] }).limit(no_of_images))

In [ ]:
print("total no of validated json: "+str(len(dishes)))

In [ ]:
data_path=PATH_DATA_ON_DISK
directory="new_data/images"
path = os.path.join(data_path, directory) 
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)
#os.makedirs(path)
directory="new_data/json"
path = os.path.join(data_path, directory) 
#os.makedirs(path)
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)

In [ ]:
#path to save the validated json and corresponding images
image_path=PATH_DATA_ON_DISK+'/new_data/images/'
json_path=PATH_DATA_ON_DISK+'/new_data/json/'

## Downloading validated json, images

In [ ]:
for i in dishes: 
    i['_id']=str(i['_id'])
    i['scanned_datetime']=str(i['scanned_datetime'])
    i['predictions']['image']['predictions_datetime']=str(i['predictions']['image']['predictions_datetime'])
   
    data=download_image(i['image_original_url'])
    img = Image.fromarray(data, 'RGB')
    #plt.imshow(img)
    img.save(image_path+i['image_original_url'].split('/')[-1])
    json_object=json.dumps(i,indent=4)
    with open(json_path+i['sample_id']+'.json','w') as outfile:
        outfile.write(json_object)

In [ ]:
#taking new data as input
images_list=[]
json_list=[]

#path of the new data
path_images = image_path+'*.*'
path_json = json_path+'/*.*'


for filename_json in glob.glob(path_json):
    json_list.append(filename_json) 
    
for filename in glob.glob(path_images):
    images_list.append(filename)


      
#sorting the array so that json and images are in same order
#note json and image should be of same name
images_list.sort()
json_list.sort()


In [ ]:
#Used to visualize a image

In [ ]:
# to visulaize the images
def plt_imshow(title, image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(title)
    plt.grid(False)
    
    plt.show()

### This below cell takes images and json from folder and crops the images based on bboxes that are getting generated from all_point_x and all_point_y from json and saving the images into two folder cfu and mold which will be later preprocessed to give input to model.

In [ ]:
parent_dir=PATH_DATA_ON_DISK
directory="data_folder/training_images/Mold"
path = os.path.join(parent_dir, directory) 
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)
#os.makedirs(path)
directory="data_folder/training_images/Cfu"
path = os.path.join(parent_dir, directory) 
#os.makedirs(path)
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)

In [ ]:
#takes image and json as input and crops only cfu's and mold's and then store in two different folder
for i in range(len(images_list)):
    img=images_list[i]
    jsn=json_list[i]    
    with open(jsn)as x:
        f = json.load(x)
    image = cv2.imread(img)
    #plt_imshow( 'original',image)
    (h, w, c) = image.shape[:3]
    #print("width: {} pixels".format(image.shape[1]))
    #print("height: {}  pixels".format(image.shape[0]))
    #print("channels: {}".format(image.shape[2]))
    #print(f['sample_id'])
    for j,item in enumerate(f["corrections"][-1]["image"]["regions"]):
        if(item['shape_attributes']['name']=='polygon'):
            x = item["shape_attributes"].get('all_points_x')
            y = item["shape_attributes"].get('all_points_y')            
            #calculating bounding boxes
            minx = int(min(x))
            maxx = int(max(x))
            miny = int(min(y))
            maxy = int(max(y))
            colony1 = image[miny:maxy+1,minx:maxx+1]
            #plt_imshow('colony1',colony1)
            #cv2.imwrite('training_images/Molds/'+filename.split()[0]+'_colony'+str(j)+'.png',colony1)
            #print(type(item['shape_attributes']['category_id']))
            if(item['shape_attributes']['category_id']==1):      
                cv2.imwrite(PATH_DATA_ON_DISK+'/data_folder/training_images/Mold/'+f['sample_id']+'_colony'+str(j)+'.png',colony1)
            else:
                cv2.imwrite(PATH_DATA_ON_DISK+'/data_folder/training_images/Cfu/'+f['sample_id']+'_colony'+str(j)+'.png',colony1)

        elif(item['shape_attributes']['name']=='circle'):
            cx = item["shape_attributes"]['cx']
            cy = item["shape_attributes"]['cy']
            r = item["shape_attributes"]['r']
            colony1 = image[int(cy-r):int(cy+r),int(cx-r):int(cx+r)]
            #plt_imshow('colony',colony1)          
            if(item['shape_attributes']['category_id']==1):
                cv2.imwrite(PATH_DATA_ON_DISK+'/data_folder/training_images/Mold/'+filename.split()[0]+'_colony'+str(j)+'.png',colony1)
            else:
                cv2.imwrite(PATH_DATA_ON_DISK+'/data_folder/training_images/Cfu/'+filename.split()[0]+'_colony'+str(j)+'.png',colony1)
print("cropped images are ready for training")

### Visualizing  cfu's and mold's

In [ ]:
#to store ratio of cfu and mold
mold_count=0
cfu_count=0
#path of the new data
path_cfu = PATH_DATA_ON_DISK+'/data_folder/training_images/Cfu/*.*'
path_mold = PATH_DATA_ON_DISK+'/data_folder/training_images/Mold/*.*'

print("Displaying some of the mold and cfu images ")
for filename in glob.glob(path_cfu):  
    cfu_count+=1 
    if cfu_count<5:
        img=filename
        image = cv2.imread(img)
        plt_imshow('cfu'+str(cfu_count),image)
        
for filename in glob.glob(path_mold):
    mold_count+=1
    if mold_count<5:
        img=filename
        image = cv2.imread(img)
        plt_imshow('mold'+str(mold_count),image)

ratio=cfu_count/mold_count
print("ratio of cfu to mold "+str(ratio))


# Data

In [ ]:
#path of cropped images
train_path =  PATH_DATA_ON_DISK+'/data_folder/training_images'
#train_path = '/hpc/projects/upt/LangModel/CFU/resnet/iteration6/test'   #zero padded images train
#valid_path = '/hpc/projects/upt/LangModel/CFU/resnet/iteration1/test'


# Resizing and Padding 

In [ ]:
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)

def resize(img,expected_size):
    return ImageOps.fit(img,expected_size,method=Image.LANCZOS)
img_height = 224
img_width = 224
def resize_images(img_height,img_width,image_list):
    for image_path in image_list:
        image = cv2.imread(str(image_path))
        img = Image.open(str(image_path))
        (h, w, c) = image.shape[:3]
        
        if (h<img_height):
            img = resize_with_padding(img, (img_height, img_width))
        else:
            img = resize(img, (img_height, img_width))
        img.save(str(image_path))
        #plt_imshow(img)
        
#data_dir = pathlib.Path("cropped image path")  #cropped image path

#cfu = list(data_dir.glob('cfu/*'))
#mold = list(data_dir.glob('mold/*'))
#custom code for taking images from local
cfu=list(glob.glob(path_cfu))
mold=list(glob.glob(path_mold))
#print(glob('training_images/Cfus/*'))




resize_images(img_height,img_width,cfu)
resize_images(img_height,img_width,mold)

In [ ]:
#%pip install split-folders

## Loading the kpis
###  From this model_kpi we will take the latest model and will load the model in saved model variable.

In [ ]:
list_models_on_blob()

#### Getting classification model kpis from Blob

In [ ]:
def get_classification_model_kpi():
    ret=get_file_from_blob('https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/model_kpi_classification.json').read()
    ret=json.loads(ret.decode('utf-8'))
    return ret  
models_kpi=get_classification_model_kpi()

In [ ]:
# storing all the version of models which will be later used to extract each model
models=models_kpi['models']

In [ ]:
#Storing latest  model name 
latest_model_name=models_kpi['models'][-1]['model_name']

In [ ]:
print("classification_models_kpi:",models_kpi)

### Downloading the latest model from blob.

In [ ]:
#downloadng the latest model
download_unzip_clean_model(model_name=latest_model_name+".zip", overwrite=False)

In [ ]:
#syntax to upload a file to blob
#upload_file_to_blob("model_kpi_classification.json",url='https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/model_kpi_classification.json',overwrite=False)

## Loading the existing model

In [ ]:
#taking model from 
try:
    #Here we are taking the latest model path from model_kpi and loading the model
    saved_model_path=models[-1]['model_path']
except :
    print("the json is not valid")
saved_model_path = PATH_MODELS_ON_DISK+"/"+latest_model_name
saved_model = tf.keras.models.load_model(saved_model_path)
model_version = int(models[-1]['model_name'].split('_')[1][0])

In [ ]:
print("current version of model is :",model_version)

# Data preparation

## Generating images with augmentation for cross validation
### Here we are creating k folder with suffled train and test images for the purpose of cross validation as well as we are augmenting the mold images as the data is highly imbalanced

In [ ]:
parent_dir=os.getcwd()
directory= PATH_DATA_ON_DISK+"/data_folder/data"
path = os.path.join(parent_dir, directory) 
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" % directory)
except OSError as error:
    print("Directory '%s' can not be created" % directory)

In [ ]:
#here k is the no of shuffled image set which will be used for k fold cross validation
k_fold=5
img_height = 224
img_width = 224

### note train_aug and val_aug is created for the model to train and test it with tta

In [ ]:
for i in range(k_fold):
    rand=random.randint(10,10000)
    #path of the image to be stored for cross validation
    output_folder= PATH_DATA_ON_DISK+"/data_folder/data/iteration"+str(i)
    splitfolders.ratio( PATH_DATA_ON_DISK+"/data_folder/training_images", output=output_folder, seed=rand, ratio=(.7, .3), group_prefix=None)
    #shutil.copytree("data_folder/data/+iteration"+str(k),"data_folder/data/iterations/iteration"+str(k))
    
    #data augmentation for mold images

    datagen =ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

    for filename in glob.glob(output_folder+"/train/Mold/*.png"):

        img = load_img(filename) 
        x = img_to_array(img) 
        x = x.reshape((1,) + x.shape) # this is a Numpy array with shape (1, 3, 299, 299)




        i = 0
        for batch in datagen.flow(x, batch_size=1,
            save_to_dir=output_folder+"/train/Mold/",
            save_prefix='mold_aug', save_format='png'):

            i += 1
            if i > int(ratio):

                break 
                # otherwise the generator would loop indefinitely

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 1.2
    #print(thresh)
    
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black" )
    fig.tight_layout()
    plt.show()
    return ax
def print_evaluation_metrics(y_pred,y_test):

    print('accuracy %s' % accuracy_score(y_pred, y_test).round(2))
    print('f1-score %s' % f1_score(y_pred, y_test,average='weighted').round(2))
    print(classification_report(y_test, y_pred,target_names = target_names))
    print(metrics.confusion_matrix(y_test,y_pred))
    print("MCC score:")
    print(round(metrics.matthews_corrcoef(y_test, y_pred),2))

# Models training

## This function returns a compiled model based on the name passed as parameter

In [ ]:
#mcc_metric = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2)
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [ ]:
def pretrained_models(model_name):
    if model_name=="inception":
        image_size=(299,299)
        
        pre_trained_model = InceptionV3(input_shape = (299, 299, 3), # Shape of our images
                                    include_top = False, # Leave out the last fully connected layer
                                    weights = 'imagenet')
        
        for layer in pre_trained_model.layers:

            layer.trainable = False
        x = layers.Flatten()(pre_trained_model.output)

        x = layers.Dense(64, activation='relu')(x)

        x = layers.Dense  (1, activation='sigmoid')(x)           

        model = Model( pre_trained_model.input, x) 

        model.compile(optimizer = 'adam', 
                      loss = 'binary_crossentropy', 
                      metrics = [keras.metrics.AUC(name='prc', curve='PR'),keras.metrics.FalseNegatives(name='fn'),keras.metrics.AUC(name='auc')])
        return model,image_size
    elif model_name =="xception":
        image_size=(299,299)
        pre_trained_model = Xception(input_shape = (299, 299, 3), # Shape of our images
                                    include_top = False, # Leave out the last fully connected layer
                                    weights = 'imagenet')
        for layer in pre_trained_model.layers:

            layer.trainable = False
        x = layers.Flatten()(pre_trained_model.output)

        x = layers.Dense(64, activation='relu')(x)

        x = layers.Dense  (1, activation='sigmoid')(x)           

        model = Model( pre_trained_model.input, x) 

        model.compile(optimizer = 'adam', 
                      loss = 'binary_crossentropy', 
                      metrics = [keras.metrics.AUC(name='prc', curve='PR'),keras.metrics.FalseNegatives(name='fn'),keras.metrics.AUC(name='auc')])
        return model,image_size
    
    elif model_name == "VGG16":
        image_size=(224,224)
        pre_trained_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
                                    include_top = False, # Leave out the last fully connected layer
                                    weights = 'imagenet')
        for layer in pre_trained_model.layers:

            layer.trainable = False
        x = layers.Flatten()(pre_trained_model.output)

        x = layers.Dense(64, activation='relu')(x)

        x = layers.Dense  (1, activation='sigmoid')(x)           

        model = Model( pre_trained_model.input, x) 

        model.compile(optimizer = 'adam', 
                      loss = 'binary_crossentropy', 
                      metrics = [keras.metrics.AUC(name='prc', curve='PR'),keras.metrics.FalseNegatives(name='fn'),keras.metrics.AUC(name='auc')])
        return model,image_size
    elif model_name == "ResNet50":
        image_size=(224,224)
        pre_trained_model = ResNet50(input_shape = (224, 224, 3), # Shape of our images
                                    include_top = False, # Leave out the last fully connected layer
                                    weights = 'imagenet')
        for layer in pre_trained_model.layers:

            layer.trainable = False
        x = layers.Flatten()(pre_trained_model.output)

        x = layers.Dense(64, activation='relu')(x)

        x = layers.Dense  (1, activation='sigmoid')(x)           

        model = Model( pre_trained_model.input, x) 

        model.compile(optimizer = 'adam', 
                      loss = 'binary_crossentropy', 
                      metrics = [keras.metrics.AUC(name='prc', curve='PR'),keras.metrics.FalseNegatives(name='fn'),keras.metrics.AUC(name='auc')])
        return model,image_size
    elif model_name == "ResNet101":
        image_size=(224,224)
        pre_trained_model = ResNet101(input_shape = (224, 224, 3), # Shape of our images
                                    include_top = False, # Leave out the last fully connected layer
                                    weights = 'imagenet')
        for layer in pre_trained_model.layers:

            layer.trainable = False
        x = layers.Flatten()(pre_trained_model.output)

        x = layers.Dense(64, activation='relu')(x)

        x = layers.Dense  (1, activation='sigmoid')(x)           

        model = Model( pre_trained_model.input, x) 

        model.compile(optimizer = 'adam', 
                      loss = 'binary_crossentropy', 
                      metrics = [keras.metrics.AUC(name='prc', curve='PR'),keras.metrics.FalseNegatives(name='fn'),keras.metrics.AUC(name='auc')])
        return model,image_size
        
        
    elif model_name == "DenseNet121":
        image_size=(224,224)
        pre_trained_model = DenseNet121(input_shape = (224, 224, 3), # Shape of our images
                                    include_top = False, # Leave out the last fully connected layer
                                    weights = 'imagenet')
        for layer in pre_trained_model.layers:

            layer.trainable = False
        
        x = layers.Flatten()(pre_trained_model.output)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(256, activation='relu'
                                )(x)
        x = layers.Dropout(0.7)(x)
        
        x = layers.BatchNormalization()(x)
        x = layers.Dense(128, activation='relu'
                                )(x)
        x = layers.Dropout(0.5)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(64, activation='relu'
                                )(x)
        x = layers.Dropout(0.3)(x)
        

        

        x = layers.Dense  (1, activation='sigmoid')(x)           

        model = Model( pre_trained_model.input, x) 

        model.compile(optimizer = 'adam', 
                      loss = 'binary_crossentropy', 
                      metrics = ['accuracy',keras.metrics.AUC(name='prc', curve='PR'),keras.metrics.FalseNegatives(name='fn'),keras.metrics.AUC(name='auc')])
        return model,image_size
        
        
    elif model_name == "Customized_CNN":
        image_size=(256,256)
        img_height=256
        img_width=256
        
        
        data_augmentation = keras.Sequential(
              [
                layers.RandomFlip("horizontal",input_shape=(img_height, img_width,1)),
                layers.RandomRotation(0.2),
                layers.RandomZoom(0.2),
                layers.RandomTranslation(0.2,0.2),
              ]
            )
        
        model = Sequential([
                  data_augmentation,
                  layers.Rescaling(1./255),
                  layers.Conv2D(16, 3, activation='relu'),
                  layers.MaxPool2D(),
                  layers.Conv2D(32, 3, activation='relu'),
                  layers.MaxPool2D(),
                  layers.Conv2D(64, 3, activation='relu'),
                  layers.MaxPool2D(),
                  layers.Flatten(),
                  layers.Dense(1024, activation='relu'),
                  layers.Dense(512, activation='relu'),
                  layers.Dense(128, activation='relu'),
                  layers.Dense(1,activation='sigmoid')
                ])
        
        model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy',keras.metrics.AUC(name='auc')])
        
        return model,image_size
        
        

In [ ]:
#path of the model where new model will be stored
#file_path=PATH_MODELS_ON_DISK+"/inception"+str(model_version+1)+"_weights.best.hdf5"
#img_width =299
#img_height =299
batch_size=32
#image_size=(299,299)


### Pass all the parameter to train model,it will do  train the model and save the model in file_path that is passed as parameter.


In [ ]:

def model_train(train_path,batch_size,image_size,model,file_path):
    
    
    train_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=image_size,
  batch_size=batch_size)
    val_ds = tf.keras.utils.image_dataset_from_directory(
  train_path,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=image_size,
  batch_size=batch_size)
    
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    
    
    
    checkpoint = ModelCheckpoint(file_path, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
#monitor mcc ,reduce lr on plateu,change mode to auto,increase epochs 1000
    early = EarlyStopping(monitor='val_auc', mode="max", patience=20)
    ReduceLR=ReduceLROnPlateau(factor=0.5, patience=5, min_lr=10e-12, verbose=1)

    callbacks_list = [checkpoint, early,ReduceLR] #early

    # fit the model
    # Run the cell. It will take some time to execute
    history = model.fit(
      train_ds,
      validation_data = val_ds,
      epochs = 300,
      verbose =True,
      callbacks = callbacks_list
    )
    return history


## Plotting the monitoring figure

In [ ]:
def plot_model_training_monitoring_figures(history):
    acc = history.history['auc']
    val_acc = history.history['val_auc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(len(val_loss))

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training auc')
    plt.plot(epochs_range, val_acc, label='Validation auc')
    plt.legend(loc='lower right')
    plt.title('Training and Validation auc')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

## Training the models

### Include the model name inside list_of_models (inception,exception,customized_cnn) to train o new data.It will also store the each models detailed report along with the mean of accuracy figure in detailed_report as and model_score respectively

In [ ]:
detailed_score={
    
}

In [ ]:
models_score={
    
}

In [ ]:
list_of_models=['VGG16']
for i in list_of_models:
    detailed_score[i]=[]
for model_name in list_of_models:
    recall=[]
    
    precision=[]
    
    mcc_score=[]
    
    accuracy=[]
    
    if (model_name in ["inception","xception"] ):
        
        img_width=299
        img_height=299
    elif (model_name in['VGG16',"ResNet50","ResNet101"]):
        img_width=224
        img_height=224
        
    elif model_name == "Customized_CNN":
        img_width=256
        img_height=256
    #note here k is the no of suffled image set (k-folled cross)
    for i in range(k_fold):
        train_path= PATH_DATA_ON_DISK+"/data_folder/data/iteration"+str(i)+"/train"
        test_path= PATH_DATA_ON_DISK+"/data_folder/data/iteration"+str(i)+"/val"
        pretrained_model,image_size = pretrained_models(model_name)
        
        # this model will be removed after the retrainig process is over
        file_path = PATH_MODELS_ON_DISK+"/"+"general_weights.best.hdf5"
        y = []
        y_pred=[]
        #history = model_train(train_path,batch_size,image_size,pretrained_model,file_path)
        if (model_name in ["inception","xception"] ):
            history = model_train(train_path,batch_size,image_size,pretrained_model,file_path)
            
            new_model = tf.keras.models.load_model(file_path)
            test_cfu_dir = test_path+"/Cfu"
            test_mold_dir = test_path+"/Mold"
            ctr=0
            test_img = []
            

            for file_name in os.listdir(test_cfu_dir):
                test_img.append(os.path.join(test_cfu_dir,file_name))
                ctr+=1
                y.append(0)
            print("no of images in cfu "+str(ctr))


            for file_name in os.listdir(test_mold_dir):
                test_img.append(os.path.join(test_mold_dir,file_name))
                ctr+=1
                y.append(1)
            print("no of images in Mold "+str(ctr))
            y = np.array(y)
            for test_image_path in test_img:

                img = tf.keras.utils.load_img(
                    test_image_path, target_size=(img_width, img_height)
                )
                img_array = keras.preprocessing.image.img_to_array(img) 
                img_array = tf.expand_dims(img_array, 0) # Create a batch
                #print(img_array.shape)

                predictions = new_model.predict(img_array)
                score = predictions[0]
                #print(score)

                if (score<0.5):

                    y_pred.append(0)
                else:

                    y_pred.append(1)
                    
        elif model_name in ["VGG16","ResNet50","ResNet101","DenseNet121"]:
            
            history = model_train(train_path,batch_size,image_size,pretrained_model,file_path)
            new_model = tf.keras.models.load_model(file_path)
            test_cfu_dir = test_path+"/Cfu"
            test_mold_dir = test_path+"/Mold"
            ctr=0
            test_img = []
            

            for file_name in os.listdir(test_cfu_dir):
                test_img.append(os.path.join(test_cfu_dir,file_name))
                ctr+=1
                y.append(0)
            print("no of images in cfu "+str(ctr))


            for file_name in os.listdir(test_mold_dir):
                test_img.append(os.path.join(test_mold_dir,file_name))
                ctr+=1
                y.append(1)
            print("no of images in Mold "+str(ctr))
            y = np.array(y)
            for test_image_path in test_img:

                img = tf.keras.utils.load_img(
                    test_image_path, target_size=(img_width, img_height)
                )
                img_array = keras.preprocessing.image.img_to_array(img) 
                img_array = tf.expand_dims(img_array, 0) # Create a batch
                #print(img_array.shape)

                predictions = new_model.predict(img_array)
                score = predictions[0]
                #print(score)

                if (score<0.5):

                    y_pred.append(0)
                else:

                    y_pred.append(1)
        
            
        elif model_name == "Customized_CNN":
            
            train_ds = tf.keras.utils.image_dataset_from_directory(
                  train_path,
                  validation_split=0.1,
                  color_mode='grayscale',
                  subset="training",
                  seed=123,
                  image_size=(img_height, img_width),
                  batch_size=batch_size)
            
            val_ds = tf.keras.utils.image_dataset_from_directory(
                      train_path,
                      validation_split=0.1,
                      color_mode='grayscale',
                      subset="validation",
                      seed=123,
                      image_size=(img_height, img_width),
                      batch_size=batch_size)

            AUTOTUNE = tf.data.AUTOTUNE

            train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
            val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)



            callbacks = [
                EarlyStopping(patience=20, verbose=1),
                ReduceLROnPlateau(factor=0.5, patience=5, min_lr=10e-12, verbose=1)
                #ModelCheckpoint(os.path.join(modelsavedir,'cfu-{epoch:03d}.hdf5'),verbose=1, save_best_only=True)
                #TensorBoard(log_dir=modelsavedir, batch_size=12, write_graph=True, write_grads=True,update_freq='batch'),
                #CSVLogger(os.path.join(modelsavedir,modelname.replace('.hdf5','.csv')), separator=',', append=False)
                ]
                               
                                   
            epochs=100
            history = pretrained_model.fit(
              train_ds,
              validation_data=val_ds,
              epochs=epochs,
               callbacks=callbacks
            )
            
            
            
            
            #saving the cnn model
            
            pretrained_model.save(file_path)
            #loading the cnn model for prediction
            new_model = tf.keras.models.load_model(file_path)
        
            #loading the test images name in a list
            
            
            ctr=0
            test_img=[]
            y = []
            test_cfu_dir = test_path+"/Cfu"
            test_mold_dir = test_path+"/Mold"
            for file_name in os.listdir(test_cfu_dir):
                test_img.append(os.path.join(test_cfu_dir,file_name))
                ctr+=1
                y.append(0)
            #print(ctr)
            ctr=0
            for file_name in os.listdir(test_mold_dir):
                test_img.append(os.path.join(test_mold_dir,file_name))
                ctr+=1
                y.append(1)
            y = np.array(y)
            #print(ctr)
    
            #from the list of test image we are predicting 
        
            for image_path in test_img:
                image = cv2.imread(str(image_path))
                
                (h, w, c) = image.shape[:3]
                
                
                img = tf.keras.utils.load_img(image_path, grayscale=True, target_size=(w, h))
                img_array = tf.keras.preprocessing.image.img_to_array(img,dtype=np.uint8) 
                orig_img = load_resized_image(img_array,img_height,img_width)
                orig_img=tf.expand_dims(orig_img , 0)
                predictions = new_model.predict(orig_img)
                score=predictions[0]
                if (score<0.5):

                    y_pred.append(0)
                else:

                    y_pred.append(1)

        
        #storing the accuracy figures mean
        recall.append(sklearn.metrics.recall_score(y,y_pred))
       
        precision.append(sklearn.metrics.precision_score(y,y_pred))
        
        mcc_score.append((metrics.matthews_corrcoef(y, y_pred)))
        
        accuracy.append(accuracy_score(y, y_pred))
        
       
        #plotting acuuracy figure for  new model
        plot_model_training_monitoring_figures(history)
        print("accuracy figure for new model with new Test data")
        
        plot_confusion_matrix(y,y_pred,classes=['cfu','mold'])
        target_names=['cfu','mold']
        print_evaluation_metrics(y_pred,y)
        cm=metrics.confusion_matrix(y,y_pred)
        detailed_score[model_name].append({"mcc_score":metrics.matthews_corrcoef(y, y_pred),
             "recall":sklearn.metrics.recall_score(y,y_pred),
             "precision":sklearn.metrics.precision_score(y,y_pred),
             "accuracy":accuracy_score(y, y_pred),
            "cfu_counted_as_mold":f"{cm[0][1]} / {cm[0][0]+cm[0][1]}",
            "mold_counted_as_cfu":f"{cm[1][0]} / {cm[1][0]+cm[1][1]}"
                                          })
    
    
    models_score[model_name]={
             "mcc_score":mean(mcc_score).round(2),
             "recall":mean(recall).round(2),
             "precision":mean(precision).round(2),
             "accuracy":mean(accuracy).round(2),
        
                             }



In [ ]:
detailed_score

## Testing of existing model on new data.


In [ ]:
recall_prev=[]

precision_prev=[]

mcc_score_prev=[]

accuracy_prev=[]

#for table purpose
mold_wrong_prev=[]
cfu_wrong_prev=[]
for i in range(k_fold):
    #train_path= PATH_DATA_ON_DISK+"/data_folder/data/iteration"+str(i)+"/train"
    test_path= PATH_DATA_ON_DISK+"/data_folder/data/iteration"+str(i)+"/val"
    #test_path= PATH_DATA_ON_DISK+"/data_folder/training_images_1"
    
    
        
    if (latest_model_name.split('_')[0] in ["inception","xception"]):
        image_size=(299,299)
        img_width=299
        img_height=299
    if (latest_model_name.split('_')[0] in ["VGG16","ResNet50","ResNet101","DenseNet121"]):
        image_size=(224,224)
        img_width=224
        img_height=224
    elif model_name == "Customized_CNN":
        img_width=256
        img_height=256
    image_size=(224,224)
    img_width=224
    img_height=224
    
    
    test_cfu_dir = test_path+"/Cfu"
    test_mold_dir = test_path+"/Mold"
    ctr=0
    test_img = []
    y = []
    y_pred=[]

    for file_name in os.listdir(test_cfu_dir):
        test_img.append(os.path.join(test_cfu_dir,file_name))
        ctr+=1
        y.append(0)
    print("no of images in cfu"+str(ctr))

    ctr=0
    for file_name in os.listdir(test_mold_dir):
        test_img.append(os.path.join(test_mold_dir,file_name))
        ctr+=1
        y.append(1)
    print("no of images in Mold"+str(ctr))
    y = np.array(y)
    y_pred_prev=[]
    
    saved_model= tf.keras.models.load_model(PATH_MODELS_ON_DISK+"/DenseNet121_iteration3.hdf5")
    for test_image_path in test_img:

        img = tf.keras.utils.load_img(
            test_image_path, target_size=(img_width, img_height)
        )
        img_array = keras.preprocessing.image.img_to_array(img) 
        img_array = tf.expand_dims(img_array, 0) # Create a batch
        #print(img_array.shape)

        
        
        score1=saved_model.predict(img_array)[0]
        if (score1<0.5):
            y_pred_prev.append(0)
        else:
            y_pred_prev.append(1)
        #print(score)

        
            
 
  
  
    recall_prev.append(sklearn.metrics.recall_score(y,y_pred_prev))
    #precision.append(sklearn.metrics.precision_score(y,y_pred))
    precision_prev.append(sklearn.metrics.precision_score(y,y_pred_prev))
    #mcc_score.append((metrics.matthews_corrcoef(y, y_pred).round(2)))
    
    
    target_names=['cfu','mold']
    #print_evaluation_metrics(y_pred,y)
    #plotting acuuracy figure for  old model
    print("accuracy figure of old model with new data")
    plot_confusion_matrix(y,y_pred_prev,classes=['cfu','mold'])
    target_names=['cfu','mold']
    print_evaluation_metrics(y_pred_prev,y)
    mcc_score_prev.append(round(metrics.matthews_corrcoef(y, y_pred_prev),2))
    #accuracy.append(accuracy_score(y,y_pred).round(2))
    accuracy_prev.append(round(accuracy_score(y,y_pred_prev),2))
    
    
    
    
    #table_purpose
    cm=metrics.confusion_matrix(y,y_pred_prev)
    
    cfu_wrong_prev.append(f"{cm[0][1]} / {cm[0][0]+cm[0][1]}")
    
    mold_wrong_prev.append(f"{cm[1][0]} / {cm[1][0]+cm[1][1]}")
    
    
    

models_score[f"existing_{latest_model_name.split('_')[0]}_model"] = {
             "mcc_score":mean(mcc_score_prev).round(2),
             "recall":mean(recall_prev).round(2),
             "precision":mean(precision_prev).round(2),
             "accuracy" : mean(accuracy_prev).round(2)
                                                                  }

In [ ]:
detailed_score[f"existing_{latest_model_name.split('_')[0]}_model"]=[]
for i in range(k_fold):
    
    detailed_score[f"existing_{latest_model_name.split('_')[0]}_model"].append({
        'mcc_score':mcc_score_prev[i],
        'recall':recall_prev[i],
        'precision':precision_prev[i],
        'accuracy':accuracy_prev[i],
        "cfu_counted_as_mold":cfu_wrong_prev[i],
        "mold_counted_as_cfu":mold_wrong_prev[i]
    })

In [ ]:
detailed_score

In [ ]:
models_score

### Now we will calculate the false positive rate and false negative rate and store it in error rate

In [ ]:
error_rate={}

In [ ]:
for i in detailed_score:
    sum_of_cfu_counted_as_mold=0
    sum_of_mold_counted_as_cfu=0
    for j in range(len(detailed_score[i])):
        
        sum_of_cfu_counted_as_mold+= int(detailed_score[i][j]['cfu_counted_as_mold'].split()[0])
        sum_of_mold_counted_as_cfu+= int(detailed_score[i][j]['mold_counted_as_cfu'].split()[0])
        #print(detailed_score[i][j]['cfu_counted_as_mold'][1])
    
    
    #print(sum_of_mold_counted_as_cfu)
    total_cfu=int(detailed_score[i][j]['cfu_counted_as_mold'].split()[2])*(len(detailed_score[i]))
    total_mold=int(detailed_score[i][j]['mold_counted_as_cfu'].split()[2])*(len(detailed_score[i]))
    false_positive_rate=sum_of_cfu_counted_as_mold/total_cfu
    false_negative_rate=sum_of_mold_counted_as_cfu/total_mold
    #print((len(detailed_score[i])))
    error_rate[i]={'false_positive_rate':round(false_positive_rate,2),'false_negative_rate':round(false_negative_rate,2)}
    

In [ ]:
error_rate

In [ ]:
models_score

## Saving Detailed Report


In [ ]:
doc=Document()
doc.add_heading('Classification Model Retraining Report',0)
doc.add_paragraph(f"Time Period between {start_date.split('T')[0]} and {end_date.split('T')[0]}")
doc.add_paragraph(f'Total no of processed images : {processed_images}')
doc.add_paragraph(f"Number of validated images : {str(len(dishes))}")
doc.add_paragraph(f"Number of Molds = {mold_count} Number of Cfus = {cfu_count}")
doc.add_paragraph(f"Train_image: Number of Molds:{math.floor(0.7*mold_count)} Number of Cfu:{math.floor(.7*cfu_count)}")
doc.add_paragraph(f"Test_image: Number of Molds:{math.ceil(0.3*mold_count)} Number of Cfu:{math.ceil(.3*cfu_count)}")
for model_name in detailed_score:
    doc.add_paragraph("Model_name : "+model_name)
    table = doc.add_table(rows=1, cols=6)
    table.style = 'Light Grid Accent 2'
    # Adding heading in the 1st row of the table
    row = table.rows[0].cells
    row[0].text = 'iteration'
    row[1].text = 'MCC_score'
    row[2].text = 'Mold_recall'
    row[3].text = 'Precision'
    row[4].text = 'Cfu predicted as Mold'
    row[5].text = 'Mold predicted as cfu'
    for j,k in enumerate(detailed_score[model_name]):
        row = table.add_row().cells
        
        
        
        row[0].text=str(j)
        row[1].text=str(round(k['mcc_score'],2))
        row[2].text=str(round(k['recall'],2))
        row[3].text=str(round(k['precision'],2))
        row[4].text=str(k['cfu_counted_as_mold'])
        row[5].text=str(k['mold_counted_as_cfu'])
    row = table.add_row().cells
    row[0].text='Mean'
    row[1].text=str(models_score[model_name]['mcc_score'])
    row[2].text=str(models_score[model_name]['recall'])
    row[3].text=str(models_score[model_name]['precision'])
    #doc.add_paragraph(f"Mean Recall :{models_score[model_name]['recall']}")
    #doc.add_paragraph(f"Mean Mcc_score :{models_score[model_name]['mcc_score']}")
    #doc.add_paragraph(f"Mean Precision :{models_score[model_name]['precision']}")
    doc.add_paragraph()
    
    
    


In [ ]:
table = doc.add_table(rows=1, cols=6)
table.style = 'Light Grid Accent 2'
# Adding heading in the 1st row of the table
row = table.rows[0].cells
row[0].text = 'Model'
row[1].text = 'Mean_MCC_score'
row[2].text = 'Mean_Mold_recall'
row[3].text = 'Mean_Precision'
row[4].text = 'False Positive rate'
row[5].text = 'False Negative rate'
for model_name in models_score:
    row = table.add_row().cells
    
    
    row[0].text=str(model_name)
    row[1].text=str(models_score[model_name]['mcc_score'])
    row[2].text=str(models_score[model_name]['recall'])
    row[3].text=str(models_score[model_name]['precision'])
    row[4].text=str(error_rate[model_name]['false_positive_rate'])
    row[5].text=str(error_rate[model_name]['false_negative_rate'])
    
    
    

In [ ]:
table.style = 'Colorful List'
doc.save(PATH_MODELS_ON_DISK+'/model_detailed_report.docx')

## Finding Best Model


In [ ]:
#we have to decide on what criteria the best model will be selected


In [ ]:
def find_best_model(models_score):
    max_recall=0
    max_mcc_score=0
    best_method=None
    
    for key,value in models_score.items():
        if   value["mcc_score"]>.9*max_mcc_score and value['recall']>=max_recall:
            max_recall=value['recall']
            max_mcc_score=value['mcc_score']
            best_method=key
            
    return(best_method)

best_model_name=find_best_model(models_score)
print(best_model_name)

In [ ]:
# path for all the images

In [ ]:
train_path=PATH_DATA_ON_DISK+"/data_folder/training_images"

In [ ]:
if(best_model_name != f"existing_{latest_model_name.split('_')[0]}_model" ):
    #train new model with whole data and update the kpi
   
    #augemnting mold data of whole data set to remove  data imbalance 

    datagen =ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
    )

    '''for filename in glob.glob(train_path+"/Mold/*.png"):

        img = load_img(filename) 
        x = img_to_array(img) 
        x = x.reshape((1,) + x.shape) # this is a Numpy array with shape (1, 3, 299, 299)




        i = 0
        for batch in datagen.flow(x, batch_size=1,
            save_to_dir=train_path+"/Mold/",
            save_prefix='mold_aug', save_format='png'):

            i += 1
            if i > int(ratio):

                break 
                # otherwise the generator would loop indefinitely'''
    
    #training a new model again with whole dataset which will be uploaded
    
    
    ##compiling the new model
    pretrained_model,image_size=pretrained_models(best_model_name)
    
    file_path = PATH_MODELS_ON_DISK+"/"+best_model_name+"_"+str(model_version+1)+".hdf5"
    
    ##training the new model with whole dataset
    history = model_train(train_path,batch_size,image_size,pretrained_model,file_path)
                 
      
    
    
    model_name=best_model_name+"_"+str(model_version+1)+".hdf5"
    #storing accuracy figure from models_score
    mcc_score=models_score[best_model_name]["mcc_score"]
    recall= models_score[best_model_name]["recall"]
    precission= models_score[best_model_name]["precision"]
    accuracy= models_score[best_model_name]["accuracy"]
    model_kpi={
        'model_name':model_name,
        'model_path':'https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/'+model_name+'.zip',
        'recall':recall,
        'precission': precission,
        'accuracy': accuracy,
        'mcc_score': mcc_score,
        'time_stamp':str(datetime.now())
    }
    print("uploading model to blob")
    zip_upload_model_to_blob(path+model_name)
    models_kpi['models'].append(model_kpi)
    with open(PATH_MODELS_ON_DISK+'/model_kpi_classification.json', 'w') as f:
        json.dump(models_kpi, f) 
    #upload the kpi
    upload_file_to_blob(PATH_MODELS_ON_DISK+'/model_kpi_classification.json',url='https://us6vxgioqcfudevsac01.blob.core.windows.net/machine-learning/models/model_kpi_classification.json',overwrite=False)
    
    

In [ ]:
#removing general saved model